# CS-GY 9223-E: Deep Learning Homework 1
Due on Sunday, 11th February 2018, 11:55 PM

This homework can be done in pairs. Everyone must submit on NYU Classes individually.

Write down the UNIs (NetIDs) of your group (if applicable)

Member 1: Titash Mandal, tm2761

Member 2: Ajay Thorve, aat414


In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import scipy.misc
import glob
import sys
# you shouldn't need to make any more imports

In [2]:
A = np.random.randn(4,3)
B = np.sum(A, axis = 1, keepdims = True)
B.shape

(4, 1)

In [95]:
class NeuralNetwork(object):
    """
    Abstraction of neural network.
    Stores parameters, activations, cached values. 
    Provides necessary functions for training and prediction. 
    """
    def __init__(self, layer_dimensions, drop_prob=0.0, reg_lambda=0.0):
        """
        Initializes the weights and biases for each layer
        :param layer_dimensions: (list) number of nodes in each layer
        :param drop_prob: drop probability for dropout layers. Only required in part 2 of the assignment
        :param reg_lambda: regularization parameter. Only required in part 2 of the assignment
        """
        np.random.seed(1)
        #As layer_dimensions is a list of the number of nodes in each layer, the length of the list is the number of 
        #hidden layers in the NN
        self.num_layers =len(layer_dimensions)
        
        #This is a dictionary which keeps W and b for each layer.
        self.parameters = {}
        self.parameters["batch_index"]=0
        for i in range(1,self.num_layers):
            #Now initializing the weights and biases for each layer
            # parameter[W1]=[...], parameter[b1]=[.....]
            self.parameters['W'+str(i)]=np.random.randn(layer_dimensions[i],layer_dimensions[i-1])*0.01
            self.parameters['b'+str(i)]=np.zeros((layer_dimensions[i],1))
            
        
        self.drop_prob = 0
        self.reg_lambda =0
        
        
        

    def affineForward(self, A, W, b):
        """
        Forward pass for the affine layer.
        :param A: input matrix, shape (L, S), where L is the number of hidden units in the previous layer and S is
        the number of samples
        :returns: the affine product WA + b, along with the cache required for the backward pass
        """
        Z= np.dot(W,A)+b
        #cache -- a python tuple containing "A", "Z" ; 
        #it is stored for computing the backward pass efficiently
        cache=(Z,A,W,b)
        return Z,cache

    def activationForward(self, A, activation="relu"):
        """
        Common interface to access all activation functions.
        :param A: input to the activation function
        :param prob: activation funciton to apply to A. Just "relu" for this assignment.
        :returns: activation(A)
        """ 
        #returns: activation(A) = relu(A)
        return self.relu(A)

    def relu(self, X):
        #ReLU is basically A=max(0,Z)
        Activation=np.maximum(0,X)
        return Activation

            
    def dropout(self, A, prob):
        """
        :param A: Activation
        :param prob: drop prob
        :returns: tuple (A, M) 
            WHERE
            A is matrix after applying dropout
            M is dropout mask, used in the backward pass
        """

        return A, M

    def forwardPropagation(self, X):
        """
        Runs an input X through the neural network to compute activations
        for all layers. Returns the output computed at the last layer along
        with the cache required for backpropagation.
        :returns: (tuple) AL, cache
            WHERE 
            AL is activation of last layer
            cache is cached values for each layer that
                     are needed in further steps
        """
        
        all_layer_cache=[]
        #for layer 1, the activation vector is basically the input vector X
        A_prev=X
        
        for i in range(1,self.num_layers):
#             print(self.parameters['W'+str(i)].shape,self.parameters['b'+str(i)].shape)
             #for each layer, we need Z,A and that's the input to the next layer.
            Z,cache = self.affineForward(A_prev, self.parameters['W'+str(i)], self.parameters['b'+str(i)])
            #Keep adding the cache of each layer to this list
            all_layer_cache.append(cache)
            #updating A_prev 
            A_prev = self.activationForward(Z, activation="relu")
        
        #AL is activation of last layer
        AL=A_prev
        print("AL",AL.shape)
#         print("cache",len(all_layer_cache[0]))
        return AL, all_layer_cache
    
    def costFunction(self, AL, y):
        """
        :param AL: Activation of last layer, shape (num_classes, S)
        :param y: labels, shape (S)
        :param alpha: regularization parameter
        :returns cost, dAL: A scalar denoting cost and the gradient of cost
        """
        # compute loss
        if(AL.shape[1]==0):
            return 0, 0
        else:
            cost = (-1/AL.shape[1]) * np.sum(np.log(AL)*y + np.log(1-AL)*(1-y))        
        #         if self.reg_lambda > 0:
        #             # add regularization
       
        
        # gradient of cost
        #What should be the values passed to relu_derivative?
        dAL = -np.divide(y,AL)+ np.divide((1-y),(1-AL))
        return cost, dAL

    def affineBackward(self, dA_prev, cache):  #dA_i and Z_i W_i, A_i, b_i
        """
        Backward pass for the affine layer.
        :param dA_prev: gradient from the next layer.
        :param cache: cache returned in affineForward
        :returns dA: gradient on the input to this layer
                 dW: gradient on the weights
                 db: gradient on the bias
        """
        #Unpacking and getting values of activation, and Z for one layer
        Z,A_prev,W,b=cache
        #m=No. of training examples
        m=A.shape[1]
        
        m = A_prev.shape[1]
        dA = np.dot(W.T,dA_prev)
        dW = (1/m) * np.dot(dA_prev, A_prev.T)
        db = (1/m) * np.sum(dA_prev, axis=1, keepdims=True)
        return dA, dW, db

    def activationBackward(self, dA, cache, activation="relu"):
        """
        Interface to call backward on activation functions.
        In this case, it's just relu. 
        """
        if activation=="relu":
            return self.relu_derivative(dA,cache)
        
        
    def relu_derivative(self, dx, cached_x):
        dx[dx<=0] = 0
        dx[dx>0]=1
        return dx
    
    def dropout_backward(self, dA, cache):

        return dA

    def backPropagation(self, dAL, Y, cache):
        """
        Run backpropagation to compute gradients on all paramters in the model
        :param dAL: gradient on the last layer of the network. Returned by the cost function.
        :param Y: labels
        :param cache: cached values during forwardprop
        :returns gradients: dW and db for each weight/bias
        """
        gradients = {}
        #Cache has an entry for each layer. So it's length tells us how many times we need to
        #Calculate the backward propagation.
        L=len(cache)
        dA_prev = dAL  # dA_L-1
        #We go from the last layer to the first layer
        for i in reversed(range(1,self.num_layers)):

            dA,dW,db=self.affineBackward(dA_prev, cache[i-1])  #dA_i, W_i, b_i, A_i
            gradients['dW'+str(i)]=dW
            gradients['db'+str(i)]=db
            dA_prev=dA
            
            
            
#             if self.drop_prob > 0:
#                 #call dropout_backward
           
            
#         if self.reg_lambda > 0:
#             # add gradients from L2 regularization to each dW
        
        return gradients


    def updateParameters(self, gradients, alpha):
        """
        :param gradients: gradients for each weight/bias
        :param alpha: step size for gradient descent 
        """
        for i in range(1,self.num_layers):
            self.parameters['W'+str(i)]=self.parameters['W'+str(i)]-(alpha*gradients['dW'+str(i)])
            self.parameters['b'+str(i)]=self.parameters['b'+str(i)]-(alpha*gradients['db'+str(i)])

    def train(self, X, y, iters=1000, alpha=0.0001, batch_size=100, print_every=100):
        """
        :param X: input samples, each column is a sample
        :param y: labels for input samples, y.shape[0] must equal X.shape[1]
        :param iters: number of training iterations
        :param alpha: step size for gradient descent
        :param batch_size: number of samples in a minibatch
        :param print_every: no. of iterations to print debug info after
        """
        
        for i in range(0, iters):
            # get minibatch
            X_batch,Y_batch=self.get_batch( X, y, batch_size)
            if(X_batch.shape[1]):
                break
            # forward prop
            #AL=predicted values of Y, through reLU
            #all_layer_cache is the cache of all layers used for back propagation.
            AL, all_layer_cache = self.forwardPropagation(X_batch)

            # compute loss
            cost, dAL = self.costFunction(AL, Y_batch)      
#             print("current cost",cost)
#             print("current dAL",dAL.shape)

            # compute gradients
            gradients=self.backPropagation(dAL, Y_batch, all_layer_cache)

            # update weights and biases based on gradient
            self.updateParameters(gradients, alpha)

            if i % print_every == 0:
                # print cost, train and validation set accuracies
                print("Cost: ", cost)
                
    def predict(self, X):
        """
        Make predictions for each sample
        """
        
        return y_pred

    def get_batch(self, X, y, batch_size):
        """
        Return minibatch of samples and labels
        
        :param X, y: samples and corresponding labels
        :parma batch_size: minibatch size
        :returns: (tuple) X_batch, y_batch
        """
        current_index=self.parameters["batch_index"]
        self.parameters["batch_index"]=self.parameters["batch_index"]+batch_size
        return X[:,current_index:current_index+batch_size], y[current_index:current_index+batch_size]

In [74]:
# Helper functions, DO NOT modify this

def get_img_array(path):
    """
    Given path of image, returns it's numpy array
    """
    return scipy.misc.imread(path)

def get_files(folder):
    """
    Given path to folder, returns list of files in it
    """
    filenames = [file for file in glob.glob(folder+'*/*')]
    filenames.sort()
    return filenames

def get_label(filepath, label2id):
    """
    Files are assumed to be labeled as: /path/to/file/999_frog.png
    Returns label for a filepath
    """
    tokens = filepath.split('/')
    label = tokens[-1].split('_')[1][:-4]
    if label in label2id:
        return label2id[label]
    else:
        sys.exit("Invalid label: " + label)

In [47]:
# Functions to load data, DO NOT change these

def get_labels(folder, label2id):
    """
    Returns vector of labels extracted from filenames of all files in folder
    :param folder: path to data folder
    :param label2id: mapping of text labels to numeric ids. (Eg: automobile -> 0)
    """
    files = get_files(folder)
    y = []
    for f in files:
        y.append(get_label(f,label2id))
    return np.array(y)

def one_hot(y, num_classes=10):

    """

    Converts each label index in y to vector with one_hot encoding

    """

    y_one_hot = np.zeros((num_classes, y.shape[0]))

    y_one_hot[y, range(y.shape[0])] = 1

    return y_one_hot

def get_label_mapping(label_file):
    """
    Returns mappings of label to index and index to label
    The input file has list of labels, each on a separate line.
    """
    with open(label_file, 'r') as f:
        id2label = f.readlines()
        id2label = [l.strip() for l in id2label]
    label2id = {}
    count = 0
    for label in id2label:
        label2id[label] = count
        count += 1
    return id2label, label2id

def get_images(folder):
    """
    returns numpy array of all samples in folder
    each column is a sample resized to 30x30 and flattened
    """
    files = get_files(folder)
    images = []
    count = 0
    
    for f in files:
        count += 1
        if count % 10000 == 0:
            print("Loaded {}/{}".format(count,len(files)))
        img_arr = get_img_array(f)
        img_arr = img_arr.flatten() / 255.0
        images.append(img_arr)
    X = np.column_stack(images)

    return X

def get_train_data(data_root_path):
    """
    Return X and y
    """
    train_data_path = data_root_path + 'train'
    id2label, label2id = get_label_mapping(data_root_path+'labels.txt')
    print(label2id)
    X = get_images(train_data_path)
    y = get_labels(train_data_path, label2id)
    return X, y

def save_predictions(filename, y):
    """
    Dumps y into .npy file
    """
    np.save(filename, y)

In [6]:
# Load the data
data_root_path = 'cifar10-hw1/'
X_train, y_train = get_train_data(data_root_path) # this may take a few minutes
X_test = get_images(data_root_path + 'test')
print('Data loading done')

{'airplane': 0, 'automobile': 1, 'bird': 2, 'cat': 3, 'deer': 4, 'dog': 5, 'frog': 6, 'horse': 7, 'ship': 8, 'truck': 9}
Loaded 10000/50000
Loaded 20000/50000
Loaded 30000/50000
Loaded 40000/50000
Loaded 50000/50000
Loaded 10000/10000
Data loading done


In [43]:
y_train.shape

(50000,)

## Part 1

#### Simple fully-connected deep neural network

In [93]:
layer_dimensions = [X_train.shape[0], 5, 10]  # including the input and output layers
NN = NeuralNetwork(layer_dimensions)
NN.train(X_train, y_train, iters=1000, alpha=0.0001, batch_size=100, print_every=100)


In [94]:
y_predicted = NN.predict(X_test)
save_predictions('ans1-uni', y_predicted)

NameError: name 'y_pred' is not defined

In [ ]:
# test if your numpy file has been saved correctly
loaded_y = np.load('ans1-uni.npy')
print(loaded_y.shape)
loaded_y[:10]

## Part 2: Regularizing the neural network
#### Add dropout and L2 regularization

In [ ]:
NN2 = NeuralNetwork(layer_dimensions, drop_prob=0, reg_lambda=0)
NN2.train(X_train, y_train, iters=1000, alpha=0.00001, batch_size=1000, print_every=10)

In [ ]:
y_predicted2 = NN2.predict(X)
save_predictions(y_predicted, 'ans2-uni')